In [7]:
###-*- coding: utf-8 -*-
"""
Created on Thu Mar 24 17:52:34 2022

@author: asutter-adc
"""

import pandas as pd

#Le fichier doit contenir impérativement les colomnes : Nom et SIREN. Les autres sont générées par le programme. 

dfile=pd.read_excel('A sireniser.xlsx')
n=len(dfile.columns.to_list())

dfile.insert(n, 'NAF', '')
dfile.insert(n+1, "Date", '')
dfile.insert(n, "Nom en capitales", '')

In [2]:
dfile


,Nom,A sireniser,Nom en capitales,NAF,Date
0,Abivax,NaN,,,
1,Fournitures Hospitalieres Industrie,NaN,,,
2,SAS Pivert,NaN,,,
3,FH Orthopedics,NaN,,,
4,Biophytis,NaN,,,
5,ICM,NaN,,,
6,Alveole,NaN,,,
7,BioNext,NaN,,,
8,Meiogenix,NaN,,,
9,Endocontrol,NaN,,,


In [3]:

#nouvelle version des fonctions d'identification et de recherche de siren, prends en compte les siren attachés. 
#Même process de recherche que test4.


from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from re import compile


def is_luhn_valid(x: int) -> bool:
    """
    Application de la formule de Luhn à un nombre
    Permet la verification du numero SIREN et Siret d'un acheteur/etablissement
    Retour:
        - bool
    """
    try:
        luhn_corr = [0, 2, 4, 6, 8, 1, 3, 5, 7, 9]
        list_number_in_x = [int(i) for i in list(str(x))]
        l2 = [luhn_corr[i] if (index + 1) % 2 == 0 else i for index, i in enumerate(list_number_in_x[::-1])]
        if sum(l2) % 10 == 0:
            return True
        elif str(x)[:9] == "356000000":  # SIREN de la Poste
            if sum(list_number_in_x) % 5 == 0:
                return True
        return False
    except:
        return False



def isSIREN(c) :
    if len(c) == 11 :
        if c[0:3].isdigit() and c[3]==' ' and c[4:7].isdigit() and c[7]==' ' and c[8:].isdigit() :
            return is_luhn_valid(''.join(c.split()))

    elif len(c) == 9 :
        return is_luhn_valid(c)

    return False

def findSIREN(chaine):

    if len(chaine) < 14 :
        return 'Pas de SIREN trouvé'

    else :
        #première étape : on recherche dans le document un SIREN sous le type ??? ??? ???
        k=0

        while k <= len(chaine)-11 :
            if isSIREN(chaine[k:k+11]) :
                return chaine[k:k+11]

            k+=1

        #deuxième étape : on recherche un numéro de TVA au format FR??SIREN
        k= 0

        while k<=len(chaine) -13 :
            if chaine[k:k+2]=='FR' and chaine[k+2:k+13].isdigit() :
                if is_luhn_valid(chaine[k+4:k+13]):
                    return chaine[k+4:k+13]

            k+=1

        #troisième étape : on recherche un SIRET sans espace dont on extrait le SIREN, avec un espace le précédant (essentiel pour éviter les longues suites de chiffres présents dans les liens)
        k=0

        while k <= len(chaine)-16 :
            if chaine[k+1:k+15].isdigit() and chaine[k]==' ':
                if is_luhn_valid(chaine[k+1:k+15]) and is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            k+=1

        #quatrième étape : on cherche un SIREN tout attaché avec espaces avant et après
        k=0

        while k<= len(chaine)-11 :

            if chaine[k]==' ' and chaine[k+1:k+10].isdigit() and chaine[k+10]==' ':
                if is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            elif chaine[k]==' ' and chaine[k+1:k+10].isdigit() and chaine[k+10]=='.':
                if is_luhn_valid(chaine[k+1:k+10]) :
                    return chaine[k+1:k+10]

            k+=1


    return 'Pas de SIREN trouvé'




In [ ]:
#construction des dictionnaires noms et nafs dont les clés sont les SIREN

i=0

noms={}
nafs={}
dates={}

siren1=open('basesiren1.txt', 'r', encoding='utf-8')
for line in siren1 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren1.close()

siren2=open('basesiren2.txt', 'r', encoding='utf-8')
for line in siren2 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren2.close()

siren3=open('basesiren3.txt', 'r', encoding='utf-8')
for line in siren3 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren3.close()

siren4=open('basesiren4.txt', 'r', encoding='utf-8')
for line in siren4 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren4.close()

siren5=open('basesiren5.txt', 'r', encoding='utf-8')
for line in siren5 :
  lst=line.strip().split(',')
  siren=lst[0]
  if lst[17]=='NAFRev2' :
    noms[siren]=lst[8]
    nafs[siren]=lst[16]
    dates[siren]=lst[1][:4]
siren5.close()



## Recherche des noms dans la base SIREN

In [22]:
# On établit tout d'abord une liste à partir de la compilation des fichiers de la base SIREN

i=0

siren=[]
noms=[]
nafs=[]
dates = []

siren1=open('basesiren1.txt', 'r', encoding='utf-8')
lines = siren1.readlines()


for line in range(len(lines)) :
    lst=lines[line].strip("\n").split(',')
    
    if lst[17]=='NAFRev2' :
        siren += [int(lst[0])]
        noms += [lst[8]]
        nafs += [lst[16]]
        dates += [lst[1][:4]]
siren1.close()

siren2=open('basesiren2.txt', 'r', encoding='utf-8')
lines = siren2.readlines()


for line in range(len(lines)) :
    lst=lines[line].strip("\n").split(',')
    if lst[17]=='NAFRev2' :
        siren += [int(lst[0])]
        noms += [lst[8]]
        nafs += [lst[16]]
        dates += [lst[1][:4]]
siren2.close()

siren3=open('basesiren3.txt', 'r', encoding='utf-8')
lines = siren3.readlines()

for line in range(len(lines)) :
    lst=lines[line].strip("\n").split(',')
    if lst[17]=='NAFRev2' :
        siren += [int(lst[0])]
        noms += [lst[8]]
        nafs += [lst[16]]
        dates += [lst[1][:4]]
siren3.close()

siren4=open('basesiren4.txt', 'r', encoding='utf-8')
lines = siren4.readlines()

for line in range(len(lines)) :
    lst=lines[line].strip("\n").split(',')
    
    if lst[17]=='NAFRev2' :
        siren += [int(lst[0])]
        noms += [lst[8]]
        nafs += [lst[16]]
        dates += [lst[1][:4]]
siren4.close()

siren5=open('basesiren5.txt', 'r', encoding='utf-8')
lines = siren5.readlines()


for line in range(len(lines)) :
    lst=lines[line].strip("\n").split(',')
    if lst[17]=='NAFRev2' :
        siren += [int(lst[0])]
        noms += [lst[8]]
        nafs += [lst[16]]
        dates += [lst[1][:4]]
siren5.close()

In [9]:
#table_siren = [siren, noms, nafs, dates]

dates

['1954',
 '1954',
 '1954',
 '1954',
 '1954',
 '1958',
 '1955',
 '1955',
 '1955',
 '1955',
 '1955',
 '1900',
 '1974',
 '1955',
 '1900',
 '1955',
 '1900',
 '1955',
 '1955',
 '1900',
 '1955',
 '1972',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1900',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1956',
 '1900',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1900',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1957',
 '1900',
 '1957',
 '1958',
 '1958',
 '1958',
 '1958',
 '1958',
 '1969',
 '1958',
 '1958',
 '1958',
 '1958',
 '1958',
 '1959',
 '1959',
 '1959',
 '1959',
 '1959',
 '1959',
 '1959',
 '1959',
 '1959',
 '1900',
 '1960',
 '1900',
 '1960',
 '1960',
 '1964',
 '1960',
 '1900',
 '1961',
 '1970',
 '1961',
 '1961',
 '1961',
 '1961',
 '1961',
 '1961',
 '1962',
 '1962',
 '1962',
 '1962',
 '1962',
 

## Envoyer la base SIREN utilisée sur un CSV simplifié

In [11]:
print(len(siren) , len(noms), len (nafs), len(dates))

# initialise data of lists.
data = {'SIREN': siren, 'Noms': noms , 'NAF' : nafs , 'date de création' : dates}
 
# Create DataFrame
data_sirene = pd.DataFrame(data)
 
# Print the output.
print(data_sirene)

8877691 8877691 8877691 8877691
             SIREN                                   Noms     NAF  \
0        005420021           ETABLISSEMENTS LUCIEN BIQUEZ  46.69B   
1        005420104                        VETEMENTS JARDI  47.71Z   
2        005420120  SOCIETE DES SUCRERIES DU MARQUENTERRE  70.10Z   
3        005450093                     BASALPINE AUTOCARS  47.78C   
4        005450119              LE GRAND CAFE ET GASSENDI  56.30Z   
...            ...                                    ...     ...   
8877686  998922108                   ETABLISSEMENTS LAITI  47.72A   
8877687  998922207      RESTAURANT DES NOUVEAUX ABATTOIRS  56.10A   
8877688  999990005                                   STEF  70.10Z   
8877689  999990021            EPARGNE ETHIQUE OBLIGATIONS  64.30Z   
8877690  999990062                             BANQUE SBA  64.19Z   

        date de création  
0                   1954  
1                   1954  
2                   1954  
3                   1954  
4   

In [ ]:
# On envoi cela sur un CSV

data_sirene.to_csv('sirene_simple.csv', sep='\t', index=False,header=True)
new_base_sirene = pd.read_csv('siren_simple.csv')
 
print(new_base_sirene)

## Construction d'un tableau sirénisé

In [5]:
# On cherche maintenant les boîtes dans notre tableau en enlevant les homonymes

def linearSearch(lys, element):
    compteur = 0
    result = -1
    for i in range (len(lys)):
        if compteur == 0 and lys[i] == element :
            result = i
            compteur += 1
        elif compteur == 1 and lys[i] == element and siren[i] != siren[result] :
            compteur += 1
            
    if compteur < 2 :    
        return result
    elif compteur > 1 :
        return -2

In [8]:
# Permet d'utiliser une colonne lisible
dfile["Nom en capitales"]= dfile["Nom"].str.upper()


In [9]:
#Opération de recherche sur la liste

column_siren = []
column_naf = []
column_dates = []
for element in dfile["Nom en capitales"] :
    number = linearSearch(noms, element)
    if number > -1 :
        column_siren += [siren[number]]
        column_naf += [nafs[number]]
        column_dates += [dates[number]]
    elif number == -2 :
        column_siren += ["HO"]
        column_naf += ["HO"]
        column_dates += ["HO"]
    else :
        column_siren += ["0"]
        column_naf += ["0"]
        column_dates += ["0"]

dfile["SIREN"] = column_siren
dfile["NAF"] = column_naf
dfile["Date"] = column_dates

In [10]:
dfile 

,Nom de la SU,A sireniser,Nom en capitales,NAF,Date,SIREN
0,Abivax,NaN,ABIVAX,72.11Z,2013,799363718
1,Fournitures Hospitalieres Industrie,NaN,FOURNITURES HOSPITALIERES INDUSTRIE,32.50A,1975,303939755
2,SAS Pivert,NaN,SAS PIVERT,0,0,0
3,FH Orthopedics,NaN,FH ORTHOPEDICS,46.46Z,1978,313620916
4,Biophytis,NaN,BIOPHYTIS,72.11Z,2006,492002225
5,ICM,NaN,ICM,HO,HO,HO
6,Alveole,NaN,ALVEOLE,HO,HO,HO
7,BioNext,NaN,BIONEXT,HO,HO,HO
8,Meiogenix,NaN,MEIOGENIX,72.19Z,2010,525187969
9,Endocontrol,NaN,ENDOCONTROL,71.12B,2006,492059217


In [11]:
dfile.to_csv('liste_sirenisee.csv', sep='\t', index=False,header=True)


## Compléter à partir des SIREN

In [23]:
#Le fichier doit contenir impérativement les colomnes : Nom et SIREN. Les autres sont générées par le programme. 

dfile=pd.read_excel('A nafer.xlsx')
n=len(dfile.columns.to_list())

dfile.insert(n, 'NAF', '')
dfile.insert(n+1, "Date", '')
dfile.insert(n, "Nom en capitales", '')

In [24]:
def linearSearch_SIREN(lys, element):
    for i in range(len(lys)):
        if lys[i] == element :
            return i
    return -1
        

In [25]:
siren

[5420021,
 5420104,
 5420120,
 5450093,
 5450119,
 5480546,
 5520028,
 5520077,
 5520176,
 5520242,
 5520325,
 5540380,
 5541552,
 5550108,
 5550157,
 5580113,
 5580121,
 5580345,
 5580436,
 5580501,
 5580618,
 5580683,
 5620034,
 5620091,
 5620117,
 5620158,
 5620174,
 5620190,
 5620281,
 5620422,
 5650031,
 5650056,
 5650080,
 5650148,
 5650189,
 5680145,
 5680236,
 5680384,
 5680541,
 5720099,
 5720164,
 5720222,
 5720313,
 5720362,
 5720537,
 5720552,
 5720594,
 5720602,
 5720610,
 5720644,
 5720651,
 5720685,
 5720750,
 5720784,
 5750153,
 5750310,
 5750377,
 5750385,
 5750559,
 5750583,
 5780044,
 5780051,
 5780226,
 5780259,
 5780283,
 5780390,
 5780515,
 5780580,
 5780960,
 5781133,
 5820030,
 5820378,
 5820428,
 5820477,
 5850011,
 5870027,
 5880257,
 5880596,
 5880638,
 5880679,
 5880737,
 5914999,
 5920038,
 5920194,
 5942875,
 5950134,
 5950159,
 5979737,
 5980016,
 5980461,
 6013742,
 6020135,
 6050074,
 6050108,
 6050116,
 6076434,
 6080014,
 6080170,
 6120042,
 6120109,


In [21]:
dfile["SIREN"]

0       859803570
1       441966504
2       498354497
3       379260656
4       819671843
          ...    
1017    883293631
1018    184401354
1019    786218974
1020    487612368
1021    450550512
Name: SIREN, Length: 1022, dtype: int64

In [27]:
# Permet d'utiliser une colonne lisible
dfile["Nom en capitales"]= dfile["Nom"].str.upper()


#Opération de recherche sur la liste

column_nom = []
column_naf = []
column_dates = []
for element in dfile["SIREN"] :
    number = linearSearch_SIREN(siren, element)
    if number > -1 :
        column_nom += [noms[number]]
        column_naf += [nafs[number]]
        column_dates += [dates[number]]
    else :
        column_nom += ["0"]
        column_naf += ["0"]
        column_dates += ["0"]

dfile["Nom en capitales"] = column_nom
dfile["NAF"] = column_naf
dfile["Date"] = column_dates

In [28]:
dfile.to_csv('liste_nafee.csv', sep='\t', index=False,header=True)